# Conseguir antlr

Para google colab:

In [1]:
# !curl -O https://www.antlr.org/download/antlr-4.13.2-complete.jar &> /dev/null

Para VSC:

In [2]:
import urllib.request

url = "https://www.antlr.org/download/antlr-4.13.2-complete.jar"
urllib.request.urlretrieve(url, "antlr-4.13.2-complete.jar")

('antlr-4.13.2-complete.jar', <http.client.HTTPMessage at 0x21b5a2be570>)

In [3]:
# se utiliza en vsc y collab

!pip install -U antlr4-python3-runtime

Defaulting to user installation because normal site-packages is not writeable


# Gramática

In [4]:
%%file trabajoparcial.g4
grammar trabajoparcial;

// Parser rules
prog: stmt* ;

stmt: expr ;

expr
    : expr '^' expr                # PowExpr
    | '-' expr                     # NegExpr
    | expr ('*' | '/' | '%') expr  # MulDivExpr
    | expr ('+' | '-') expr        # AddSubExpr
    | func '(' expr ')'            # FuncExpr
    | integral                     # IntegralExpr
    | integralIndef                # IntegralIndefExpr
    | deriv                        # DerivExpr
    | derivDef                     # DerivDefExpr
    | NUM                          # NumExpr
    | ID                           # IdExpr
    | '(' expr ')'                 # ParenExpr
    ;

// Funciones matemáticas soportadas
func: FUNC ;

INTEGRAL: 'integral';
DERIV: 'deriv';
FUNC: 'sin' | 'cos' | 'tan' | 'log' | 'exp';
ID    : [a-zA-Z_][a-zA-Z_0-9]* ;
NUM   : [0-9]+ ('.' [0-9]+)? ;

// Definite integral: integral(a, b, expr)
integral: INTEGRAL LPAR expr ',' expr ',' expr RPAR ;

// Indefinite integral: integral(expr, var)
integralIndef: INTEGRAL LPAR expr ',' ID RPAR ;

// Derivative: deriv(expr, var)
deriv: DERIV LPAR expr ',' ID RPAR ;

// Definite derivative: deriv(a, b, expr, var)
derivDef: DERIV LPAR expr ',' expr ',' expr ',' ID RPAR ;

ADD   : '+' ;
SUB   : '-' ;
MUL   : '*' ;
DIV   : '/' ;
MOD   : '%' ;
POW   : '^' ;
LPAR  : '(' ;
RPAR  : ')' ;

WS    : [ \t\r\n]+ -> skip ;

Overwriting trabajoparcial.g4


In [5]:
%%file ej1.xyz
sin(3.14/2)
integral(0, 1, x^2 + 2*x + 1)
integral(x^2 + 2*x + 1, x)
deriv(x^2 + 2*x + 1, x)
deriv(0, 1, x^2 + 2*x + 1, x)
sin(deriv(x^2, x)) + integral(0, 2, x)

Overwriting ej1.xyz


# Crear archivos para el entorno de antlr

Para google collab

In [6]:
# !java -cp .:antlr-4.13.2-complete.jar org.antlr.v4.Tool trabajoparcial.g4 -Dlanguage=Python3

Para VSC

Instalar Java. [página](https://www.oracle.com/java/technologies/downloads/#jdk25-windows) y seleccionamos el [instalador](https://download.oracle.com/java/25/latest/jdk-25_windows-x64_bin.exe ). Cambiamos el : por ;

In [7]:
!java -cp .;antlr-4.13.2-complete.jar org.antlr.v4.Tool trabajoparcial.g4 -Dlanguage=Python3

# Ejecucción

In [8]:
from antlr4 import *
from trabajoparcialLexer import trabajoparcialLexer
from trabajoparcialParser import trabajoparcialParser

In [9]:
lexer = trabajoparcialLexer(FileStream('ej1.xyz'))
stream = CommonTokenStream(lexer)
parser = trabajoparcialParser(stream)
tree = parser.prog()
strTree = tree.toStringTree(recog=parser)
print(strTree)

(prog (stmt (expr (func sin) ( (expr (expr 3.14) / (expr 2)) ))) (stmt (expr (integral integral ( (expr 0) , (expr 1) , (expr (expr (expr (expr x) ^ (expr 2)) + (expr (expr 2) * (expr x))) + (expr 1)) )))) (stmt (expr (integralIndef integral ( (expr (expr (expr (expr x) ^ (expr 2)) + (expr (expr 2) * (expr x))) + (expr 1)) , x )))) (stmt (expr (deriv deriv ( (expr (expr (expr (expr x) ^ (expr 2)) + (expr (expr 2) * (expr x))) + (expr 1)) , x )))) (stmt (expr (derivDef deriv ( (expr 0) , (expr 1) , (expr (expr (expr (expr x) ^ (expr 2)) + (expr (expr 2) * (expr x))) + (expr 1)) , x )))) (stmt (expr (expr (func sin) ( (expr (deriv deriv ( (expr (expr x) ^ (expr 2)) , x ))) )) + (expr (integral integral ( (expr 0) , (expr 2) , (expr x) ))))))


In [10]:
from trabajoparcialListener import trabajoparcialListener

class MyListener(trabajoparcialListener):

    def enterProg(self, ctx:trabajoparcialParser.ProgContext):
        print("Entramos a la regla prog")

    def exitProg(self, ctx:trabajoparcialParser.ProgContext):
        print("Salimos de la regla prog")

    def enterStmt(self, ctx:trabajoparcialParser.StmtContext):
        print("Entramos a la regla stmt")

    def exitStmt(self, ctx:trabajoparcialParser.StmtContext):
        print("Saliendo de la regla stmt")

    def enterExpr(self, ctx:trabajoparcialParser.ExprContext):
        print("Entramos a la regla expr", ctx.getText())

    def exitExpr(self, ctx:trabajoparcialParser.ExprContext):
        print("Saliendo de la regla expr")

In [11]:
lexer = trabajoparcialLexer(FileStream('ej1.xyz'))
stream = CommonTokenStream(lexer)
parser = trabajoparcialParser(stream)
tree = parser.prog()
walker = ParseTreeWalker()
walker.walk(MyListener(), tree);

Entramos a la regla prog
Entramos a la regla stmt
Saliendo de la regla stmt
Entramos a la regla stmt
Saliendo de la regla stmt
Entramos a la regla stmt
Saliendo de la regla stmt
Entramos a la regla stmt
Saliendo de la regla stmt
Entramos a la regla stmt
Saliendo de la regla stmt
Entramos a la regla stmt
Saliendo de la regla stmt
Salimos de la regla prog


# Visitor

In [12]:
!java -cp .;antlr-4.13.2-complete.jar org.antlr.v4.Tool trabajoparcial.g4 -no-listener -visitor -Dlanguage=Python3

In [13]:
# Implementación básica de Visitor para la gramática trabajoparcial
from antlr4 import *
from trabajoparcialLexer import trabajoparcialLexer
from trabajoparcialParser import trabajoparcialParser
from trabajoparcialVisitor import trabajoparcialVisitor

In [14]:
%%writefile Visitor.py
if __name__ is not None and "." in __name__:
    from .trabajoparcialParser import trabajoparcialParser
    from .trabajoparcialVisitor import trabajoparcialVisitor
else:
    from trabajoparcialParser import trabajoparcialParser
    from trabajoparcialVisitor import trabajoparcialVisitor
    
class Visitor(trabajoparcialVisitor):
    def visitProg(self, ctx):
        print("visitProg")
        for stmt in ctx.stmt():
            self.visit(stmt)

    def visitStmt(self, ctx):
        print("visitStmt")
        resultado = self.visit(ctx.expr())
        print(f"Resultado de la expresión: {resultado}")
        
    def visitExprAction(self, ctx):
        resultado = self.visit(ctx.expr())
        print(resultado)
        return resultado

    def visitNumExpr(self, ctx):
        print(f"visitNumExpr: {ctx.NUM().getText()}")
        return float(ctx.NUM().getText())

    def visitIdExpr(self, ctx):
        print(f"visitIdExpr: {ctx.ID().getText()}")
        return ctx.ID().getText()

    def visitAddSubExpr(self, ctx):
        left = self.visit(ctx.expr(0))
        right = self.visit(ctx.expr(1))
        op = ctx.getChild(1).getText()
        resultado = left + right if op == '+' else left - right
        print(f"Evaluando: {left} {op} {right} = {resultado}")
        return resultado

    def visitMulDivExpr(self, ctx):
        left = self.visit(ctx.expr(0))
        right = self.visit(ctx.expr(1))
        op = ctx.getChild(1).getText()
        if op == '*':
            resultado = left * right
        elif op == '/':
            resultado = left / right
        else:
            resultado = left % right
        print(f"Evaluando: {left} {op} {right} = {resultado}")
        return resultado

    def visitPowExpr(self, ctx):
        left = self.visit(ctx.expr(0))
        right = self.visit(ctx.expr(1))
        resultado = left ** right
        print(f"Evaluando: {left} ^ {right} = {resultado}")
        return resultado

    def visitNegExpr(self, ctx):
        valor = self.visit(ctx.expr())
        resultado = -valor
        print(f"Evaluando: -{valor} = {resultado}")
        return resultado

    def visitParenExpr(self, ctx):
        print("visitParenExpr")
        return self.visit(ctx.expr())

    def visitFuncExpr(self, ctx):
        import math
        func = ctx.func().getText()
        arg = self.visit(ctx.expr())
        if func == 'sin':
            resultado = math.sin(arg)
        elif func == 'cos':
            resultado = math.cos(arg)
        elif func == 'tan':
            resultado = math.tan(arg)
        elif func == 'log':
            resultado = math.log(arg)
        elif func == 'exp':
            resultado = math.exp(arg)
        else:
            raise Exception(f'Función no soportada: {func}')
        print(f"Evaluando: {func}({arg}) = {resultado}")
        return resultado

    def visitIntegralExpr(self, ctx):
        print("visitIntegralExpr")
        # Acceso correcto a los hijos según la gramática
        a = self.visit(ctx.expr(0))
        b = self.visit(ctx.expr(1))
        expr = ctx.expr(2).getText()
        print(f'Integral definida de {expr} desde {a} hasta {b}')
        return None

    def visitIntegralIndefExpr(self, ctx):
        print("visitIntegralIndefExpr")
        expr = ctx.expr(0).getText()
        var = ctx.ID().getText()
        print(f'Integral indefinida de {expr} respecto a {var}')
        return None

    def visitDerivExpr(self, ctx):
        print("visitDerivExpr")
        expr = ctx.expr(0).getText()
        var = ctx.ID().getText()
        print(f'Derivada de {expr} respecto a {var}')
        return None

    def visitDerivDefExpr(self, ctx):
        print("visitDerivDefExpr")
        a = self.visit(ctx.expr(0))
        b = self.visit(ctx.expr(1))
        expr = ctx.expr(2).getText()
        var = ctx.ID().getText()
        print(f'Derivada definida de {expr} respecto a {var} entre {a} y {b}')
        return None


Overwriting Visitor.py


In [15]:
%%writefile ej1.xyz
sin(3.14/2)

Overwriting ej1.xyz


In [18]:
from antlr4 import *
from trabajoparcialLexer import trabajoparcialLexer
from trabajoparcialParser import trabajoparcialParser

from Visitor import Visitor
import sys

input_stream = FileStream('ej1.xyz')

lexer = trabajoparcialLexer(input_stream)
token_stream = CommonTokenStream(lexer)
parser = trabajoparcialParser(token_stream)

tree = parser.prog()

visitor = Visitor()
visitor.visit(tree)